# Create a syntetic dataset

In [46]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
import math
from multiprocessing import  Pool

In [47]:
FMT = 4
N = 1000

In [48]:
# Define two tweets that represent two topics
topic1 = remove_stopwords("parmesan is the best italian food ingredient my food life consists of italian parmesan tomatoes beef and parmesan lets eat all the food")
topic1Period = 7
topic1Offset = 0

topic2 = remove_stopwords("football is my life kick a ball in the goal the goal is to win win win sports football is a sports lets kick")
topic2Period = 15
topic2Offset = 4

In [49]:
# Import dataset without patterns
df = pd.read_csv('1000EnForSynth.csv', encoding='cp1252')
df = df.drop(["Unnamed: 0"], axis=1)
df

,timestamp_ms,longitude,latitude,text,name,admin1,cc
0,1443887442007,-66.578926,6.422820,quiero que sea sponsor zayn malik goals rts sa...,Puerto Carreno,Vichada,CO
1,1443887443914,-73.948775,40.655138,confirmed justin bieber incredibly dumb,Brooklyn,New York,US
2,1443887588840,-122.630908,45.536402,going taco bell absolutely makes taco,Portland,Oregon,US
3,1443888294480,-100.076888,31.168893,atlanta backwards atlanta,Eden,Texas,US
4,1443888510207,-89.266507,39.739300,republican strategist actually said morning ch...,Edinburg,Illinois,US
5,1443888522674,-73.948775,40.655138,wouldn analogy sense went church eat christians,Brooklyn,New York,US
6,1443888651009,-71.112377,42.378197,pluralistic morally superior position foreign ...,Cambridge,Massachusetts,US
7,1443888724091,-89.266507,39.739300,concludes gun safety laws work sorry chicago b...,Edinburg,Illinois,US
8,1443888866077,-89.266507,39.739300,easy guns sorry strategist deeper trouble thou...,Edinburg,Illinois,US
9,1443888999046,-76.069946,36.802541,way timestamps imessages,Virginia Beach,Virginia,US


In [35]:
# Insert periodic topics
# Topic 1: Insert at two locations
# Set topic text
# Store rows with location info per location
row1loc1 = df.iloc[topic1Offset*FMT]
row1loc2 = df.iloc[98]
row1loc1['text'] = topic1
row1loc2['text'] = topic1
# Set rows periodically to contain the same text and location
for i in range(topic1Offset*FMT, N, topic1Period*FMT):
    df.iloc[i] = row1loc1
    df.iloc[i+1] = row1loc1
    
    df.iloc[i+FMT] = row1loc2
    df.iloc[i+FMT+1] = row1loc2


# Topic 2: Insert at four locations, same procedure as topic 1. 
# Set in the last two timestamps per day to avoid crashing with topic 1.
row2loc1 = df.iloc[11]
row2loc2 = df.iloc[272]
row2loc3 = df.iloc[21]
row2loc4 = df.iloc[485]

row2loc1['text'] = topic2
row2loc2['text'] = topic2
row2loc3['text'] = topic2
row2loc4['text'] = topic2

for i in range(topic2Offset * FMT, N, topic2Period * FMT):
    try:
        df.iloc[i+2] = row2loc1
        df.iloc[i+3] = row2loc1

        df.iloc[i+(2*FMT)+2] = row2loc2
        df.iloc[i+(2*FMT)+3] = row2loc2

        df.iloc[i+(3*FMT)+2] = row2loc3
        df.iloc[i+(3*FMT)+3] = row2loc3

        df.iloc[i+(7*FMT)+2] = row2loc4
        df.iloc[i+(7*FMT)+3] = row2loc4
    except:
        print(i)

<ipython-input-35-2588ce3ccd19>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row1loc1['text'] = topic1
<ipython-input-35-2588ce3ccd19>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row1loc2['text'] = topic1
<ipython-input-35-2588ce3ccd19>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row2loc1['text'] = topic2
<ipython-input-35-2588ce3ccd19>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

976


In [50]:
# Set new timestamps, generate FMT timestamps per day, frequency = 24/FMTH. Convert to milliseconds
freq = str(24//FMT) + "H"
dti = pd.date_range("2018-01-01", periods=N, freq=freq)
df["timestamp_date"] = dti
df['timestamp_ms'] = df['timestamp_date'].astype(np.int64) / int(1e6)
df.timestamp_ms = df.timestamp_ms.astype(int)
df = df.drop(['timestamp_date'], axis=1)

In [51]:
df

,timestamp_ms,longitude,latitude,text,name,admin1,cc
0,1514764800000,-66.578926,6.422820,quiero que sea sponsor zayn malik goals rts sa...,Puerto Carreno,Vichada,CO
1,1514786400000,-73.948775,40.655138,confirmed justin bieber incredibly dumb,Brooklyn,New York,US
2,1514808000000,-122.630908,45.536402,going taco bell absolutely makes taco,Portland,Oregon,US
3,1514829600000,-100.076888,31.168893,atlanta backwards atlanta,Eden,Texas,US
4,1514851200000,-89.266507,39.739300,republican strategist actually said morning ch...,Edinburg,Illinois,US
5,1514872800000,-73.948775,40.655138,wouldn analogy sense went church eat christians,Brooklyn,New York,US
6,1514894400000,-71.112377,42.378197,pluralistic morally superior position foreign ...,Cambridge,Massachusetts,US
7,1514916000000,-89.266507,39.739300,concludes gun safety laws work sorry chicago b...,Edinburg,Illinois,US
8,1514937600000,-89.266507,39.739300,easy guns sorry strategist deeper trouble thou...,Edinburg,Illinois,US
9,1514959200000,-76.069946,36.802541,way timestamps imessages,Virginia Beach,Virginia,US


In [52]:
fmt = '%d\n%.8f\n%.8f\n%s\n%s\n%s\n%s'
np.savetxt(r'datasets/datasetSynth1000_1_nopatterns.txt', df.values, fmt=fmt, delimiter='\r\n')

In [39]:
row1loc1

timestamp_ms                                        1443887442007
longitude                                                -66.5789
latitude                                                  6.42282
text            parmesan best italian food ingredient food lif...
name                                               Puerto Carreno
admin1                                                    Vichada
cc                                                             CO
Name: 0, dtype: object

In [40]:
row1loc2

timestamp_ms                                        1443893461924
longitude                                                 9.98366
latitude                                                  53.5506
text            parmesan best italian food ingredient food lif...
name                                                     Neustadt
admin1                                                    Hamburg
cc                                                             DE
Name: 98, dtype: object

In [41]:
row2loc1

timestamp_ms                                        1443889281097
longitude                                                  121.45
latitude                                                  14.2528
text            football life kick ball goal goal win win win ...
name                                                   Cabanbanan
admin1                                                 Calabarzon
cc                                                             PH
Name: 11, dtype: object

In [42]:
row2loc2

timestamp_ms                                        1443903723223
longitude                                                -60.0298
latitude                                                 -37.1476
text            football life kick ball goal goal win win win ...
name                                                    Olavarria
admin1                                               Buenos Aires
cc                                                             AR
Name: 272, dtype: object

In [43]:
row2loc3

timestamp_ms                                        1443890524089
longitude                                                  120.97
latitude                                                  14.4305
text            football life kick ball goal goal win win win ...
name                                                    Las Pinas
admin1                                                 Calabarzon
cc                                                             PH
Name: 21, dtype: object

In [44]:
row2loc4

timestamp_ms                                        1443911709978
longitude                                                -79.2726
latitude                                                  43.6293
text            football life kick ball goal goal win win win ...
name                                                  Scarborough
admin1                                                    Ontario
cc                                                             CA
Name: 485, dtype: object